In [7]:
import os
import numpy as np
import pandas as pd
from collections import Counter 
from keras.models import Model
from keras.layers import Input, Embedding, BatchNormalization,  LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
from preproc import build_vocabulary, Vectorizer 

input_vec_len1 = 20
input_vec_len2 = 30 

num_labels1  = 4
num_labels2  = 6 

latent_dim  = 60 
num_tokens1 = 100
num_tokens2 = 120 


def get_model ():
    
    input1  = Input(shape=(input_vec_len1,),  name='Input-Layer1')
    x1 = Embedding(num_tokens1, latent_dim, name='Embedding-Layer1', mask_zero=False) (input1)
    x1 = BatchNormalization(name='Encoder-Batchnorm-1')(x1)
    y1, h1, c1 =  LSTM(latent_dim, return_state=True) (x1)

    input2  = Input(shape=(input_vec_len2,),  name='Input-Layer2')
    x2 = Embedding(num_tokens2, latent_dim, name='Embedding-Layer2', mask_zero=False) (input2)
    x2 = BatchNormalization(name='Encoder-Batchnorm-2')(x2)
    y2, h2, c2 =  LSTM(latent_dim, return_state=True) (x2)

    y = y1 + y1
    h = h1 + h2  
    
    final_output1 = Dense(num_labels1, activation='softmax', name='Final-Output-Dense1')(y)
    final_output2 = Dense(num_labels2, activation='softmax', name='Final-Output-Dense2')(h)

    model = Model(inputs = [input1,input2], outputs=[final_output1,final_output2], name='Encoder-Model')
    return model 
    
    
if __name__ == "__main__":
    M = get_model ()
    print(M.summary())
   



Model: "Encoder-Model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input-Layer1 (InputLayer)   [(None, 20)]                 0         []                            
                                                                                                  
 Input-Layer2 (InputLayer)   [(None, 30)]                 0         []                            
                                                                                                  
 Embedding-Layer1 (Embeddin  (None, 20, 60)               6000      ['Input-Layer1[0][0]']        
 g)                                                                                               
                                                                                                  
 Embedding-Layer2 (Embeddin  (None, 30, 60)               7200      ['Input-Layer2[0][